In [1]:
import numpy  as np  
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import time
from keras import models, layers, optimizers, regularizers
import warnings
warnings.filterwarnings('ignore')

# 2.-  Modelo para predecir el precio con Redes Neuronales usando solo datos numéricos (sin imágenes)

#### Funciones auxiliares

In [2]:
def compile(modelo):
    modelo.compile(loss='mean_squared_error',
                   optimizer='adam',
                   metrics=['mean_squared_error'])

In [3]:
def evaluacion(X_train, y_train, X_test, y_test, modelo):
  #Evaluo el modelo con las muestras de test
  y_test_pred = modelo.predict(X_test)
  y_train_pred = modelo.predict(X_train)
  print("Training MSE:", round(mean_squared_error(y_train, y_train_pred),4))
  print("Validation MSE:", round(mean_squared_error(y_test, y_test_pred),4))
  print("\nTraining r2:", round(r2_score(y_train, y_train_pred),4))
  print("Validation r2:", round(r2_score(y_test, y_test_pred),4))

In [4]:
def entrenamiento(X_train, y_train, e, b, v, modelo): 
  #Entreno el modelo con 100 épocas y batch_size de 256
  modelo_start = time.time()
  modelo.fit(X_train,
            y_train,
            epochs = e,
            batch_size = b,
            validation_split = v)  #Con esto se separa una porción del conjunto de entratamiento y se usa como
                                            #validación, con esa porción no se entrena                  
  modelo_end = time.time()
  print(f"Tiempo de ejecución: {round((modelo_end - modelo_start)/60,1)} minutes")

#### A continuación leo los datos limpios y los preparo para applicar diferentes modelos

In [5]:
train = pd.read_csv('Data/train_cleaned.csv', sep =',', decimal='.', index_col=0) 
test = pd.read_csv('Data/test_cleaned.csv', sep =',', decimal='.', index_col=0) 

Para este primer modelado no me hacen faltas las columnas "ID" y "Picture Url" así que las voy a eliminar

In [6]:
train.drop (['ID', 'Picture Url'], axis=1, inplace=True)
test.drop (['ID', 'Picture Url'], axis=1, inplace=True)

 Estandarización y Normalización

In [7]:
# Transformación logarítmica de columnas
numerical_columns = [i for i in train.columns if i not in ['Availability 30']] # Quito las que no se van a transformar

for col in numerical_columns:
    train[col] = train[col].astype('float64').replace(0.0, 0.01) # Replacing 0s with 0.01
    train[col] = np.log(train[col])
    test[col] = test[col].astype('float64').replace(0.0, 0.01) # Replacing 0s with 0.01
    test[col] = np.log(test[col])

Separamos las variables predictivas X de la variable objetivo y escalamos X. 

In [8]:
# Separamos X and y
X_train = train.drop('Price', axis=1)
y_train = train.Price

X_test = test.drop('Price', axis=1)
y_test = test.Price

# Scaling
scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=list(X_train.columns))

X_test = pd.DataFrame(scaler.transform(X_test), columns=list(X_test.columns))

## 2.1. Modelo 1: Red neuronal con 3 capas
Voy a comenzar con una red neuronal de 3 capas. Utilizaré 3 capas densas, una fución de activación de tipo relu y para la salida utilizaré una función lineal al ser un problema de regresión. La función de pérdidas será la "mean squered_error'. Utilizaré además como optimizador adam con su lr por defecto

In [9]:
#Comprobamos el número de características. La primera capa debe tener más neuronas.
len  (list(X_train.columns))

10

In [10]:
#Creo el modelo
modelo = models.Sequential()
modelo.add (layers.Dense (16, input_shape=(X_train.shape[1],), activation='relu'))
modelo.add(layers.Dense(32, activation='relu'))
modelo.add(layers.Dense(32, activation='relu'))
modelo.add(layers.Dense(1, activation='linear'))

#Compilo el modelo 
compile(modelo)
print(modelo.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 16)                176       
_________________________________________________________________
dense_1 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_2 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 1,809
Trainable params: 1,809
Non-trainable params: 0
_________________________________________________________________
None


In [11]:
#Entreno el modelo con 100 épocas y batch_size de 256
entrenamiento(X_train, y_train, 100, 256, 0.1, modelo)

#Evaluo el modelo con las muestras de test
evaluacion(X_train, y_train, X_test, y_test, modelo)

Epoch 1/100
37/37 [==============================] - 1s 5ms/step - loss: 0.2524 - mean_squared_error: 0.2524 - val_loss: 0.0571 - val_mean_squared_error: 0.0571
Epoch 2/100
37/37 [==============================] - 0s 1ms/step - loss: 0.0394 - mean_squared_error: 0.0394 - val_loss: 0.0252 - val_mean_squared_error: 0.0252
Epoch 3/100
37/37 [==============================] - 0s 1ms/step - loss: 0.0208 - mean_squared_error: 0.0208 - val_loss: 0.0167 - val_mean_squared_error: 0.0167
Epoch 4/100
37/37 [==============================] - 0s 1ms/step - loss: 0.0154 - mean_squared_error: 0.0154 - val_loss: 0.0138 - val_mean_squared_error: 0.0138
Epoch 5/100
37/37 [==============================] - 0s 1ms/step - loss: 0.0133 - mean_squared_error: 0.0133 - val_loss: 0.0122 - val_mean_squared_error: 0.0122
Epoch 6/100
37/37 [==============================] - 0s 1ms/step - loss: 0.0120 - mean_squared_error: 0.0120 - val_loss: 0.0111 - val_mean_squared_error: 0.0111
Epoch 7/100
37/37 [===============

37/37 [==============================] - 0s 1ms/step - loss: 0.0078 - mean_squared_error: 0.0078 - val_loss: 0.0082 - val_mean_squared_error: 0.0082
Epoch 52/100
37/37 [==============================] - 0s 1ms/step - loss: 0.0078 - mean_squared_error: 0.0078 - val_loss: 0.0080 - val_mean_squared_error: 0.0080
Epoch 53/100
37/37 [==============================] - 0s 1ms/step - loss: 0.0078 - mean_squared_error: 0.0078 - val_loss: 0.0080 - val_mean_squared_error: 0.0080
Epoch 54/100
37/37 [==============================] - 0s 1ms/step - loss: 0.0077 - mean_squared_error: 0.0077 - val_loss: 0.0081 - val_mean_squared_error: 0.0081
Epoch 55/100
37/37 [==============================] - 0s 1ms/step - loss: 0.0077 - mean_squared_error: 0.0077 - val_loss: 0.0080 - val_mean_squared_error: 0.0080
Epoch 56/100
37/37 [==============================] - 0s 1ms/step - loss: 0.0077 - mean_squared_error: 0.0077 - val_loss: 0.0080 - val_mean_squared_error: 0.0080
Epoch 57/100
37/37 [=====================

Para ser un primer intento da un resultado bastante aceptable. Sobre todo en cuanto al overfitting

## 2.2. Modelo 2: Red neuronal con 3 capas, regularización L1 y más épocas
Usamos regularización L1. Añadimos una capa adicional para mejorar el resultado (r2)

In [12]:
# Construyo el modelo
modelo2 = models.Sequential()
modelo2.add(layers.Dense(16, input_shape=(X_train.shape[1],), kernel_regularizer=regularizers.l1(0.005), activation='relu'))
modelo2.add(layers.Dense(32, kernel_regularizer=regularizers.l1(0.005), activation='relu'))
modelo2.add(layers.Dense(32, kernel_regularizer=regularizers.l1(0.005), activation='relu'))
modelo2.add(layers.Dense(1, activation='linear'))

# Compilo el modelo
compile(modelo2)

# Model summary
print(modelo2.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_4 (Dense)              (None, 16)                176       
_________________________________________________________________
dense_5 (Dense)              (None, 32)                544       
_________________________________________________________________
dense_6 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 33        
Total params: 1,809
Trainable params: 1,809
Non-trainable params: 0
_________________________________________________________________
None


In [13]:
#Entreno el modelo con 150 épocas y batch_size de 256
entrenamiento(X_train, y_train, 150, 256, 0.1, modelo2)

#Evaluo el modelo con las muestras de test
evaluacion(X_train, y_train, X_test, y_test, modelo2)

Epoch 1/150
37/37 [==============================] - 1s 5ms/step - loss: 1.4223 - mean_squared_error: 0.0661 - val_loss: 1.2676 - val_mean_squared_error: 0.0255
Epoch 2/150
37/37 [==============================] - 0s 1ms/step - loss: 1.1508 - mean_squared_error: 0.0213 - val_loss: 1.0283 - val_mean_squared_error: 0.0155
Epoch 3/150
37/37 [==============================] - 0s 1ms/step - loss: 0.9236 - mean_squared_error: 0.0144 - val_loss: 0.8147 - val_mean_squared_error: 0.0117
Epoch 4/150
37/37 [==============================] - 0s 1ms/step - loss: 0.7233 - mean_squared_error: 0.0120 - val_loss: 0.6285 - val_mean_squared_error: 0.0108
Epoch 5/150
37/37 [==============================] - 0s 2ms/step - loss: 0.5496 - mean_squared_error: 0.0116 - val_loss: 0.4690 - val_mean_squared_error: 0.0111
Epoch 6/150
37/37 [==============================] - 0s 1ms/step - loss: 0.4038 - mean_squared_error: 0.0121 - val_loss: 0.3372 - val_mean_squared_error: 0.0116
Epoch 7/150
37/37 [===============

37/37 [==============================] - 0s 1ms/step - loss: 0.0179 - mean_squared_error: 0.0095 - val_loss: 0.0174 - val_mean_squared_error: 0.0090
Epoch 52/150
37/37 [==============================] - 0s 1ms/step - loss: 0.0178 - mean_squared_error: 0.0095 - val_loss: 0.0174 - val_mean_squared_error: 0.0092
Epoch 53/150
37/37 [==============================] - 0s 1ms/step - loss: 0.0178 - mean_squared_error: 0.0095 - val_loss: 0.0173 - val_mean_squared_error: 0.0089
Epoch 54/150
37/37 [==============================] - 0s 1ms/step - loss: 0.0177 - mean_squared_error: 0.0095 - val_loss: 0.0173 - val_mean_squared_error: 0.0091
Epoch 55/150
37/37 [==============================] - 0s 1ms/step - loss: 0.0177 - mean_squared_error: 0.0095 - val_loss: 0.0172 - val_mean_squared_error: 0.0090
Epoch 56/150
37/37 [==============================] - 0s 1ms/step - loss: 0.0177 - mean_squared_error: 0.0095 - val_loss: 0.0171 - val_mean_squared_error: 0.0090
Epoch 57/150
37/37 [=====================

37/37 [==============================] - 0s 1ms/step - loss: 0.0167 - mean_squared_error: 0.0094 - val_loss: 0.0163 - val_mean_squared_error: 0.0090
Epoch 102/150
37/37 [==============================] - 0s 1ms/step - loss: 0.0168 - mean_squared_error: 0.0094 - val_loss: 0.0163 - val_mean_squared_error: 0.0090
Epoch 103/150
37/37 [==============================] - 0s 1ms/step - loss: 0.0167 - mean_squared_error: 0.0094 - val_loss: 0.0163 - val_mean_squared_error: 0.0090
Epoch 104/150
37/37 [==============================] - 0s 1ms/step - loss: 0.0167 - mean_squared_error: 0.0094 - val_loss: 0.0162 - val_mean_squared_error: 0.0089
Epoch 105/150
37/37 [==============================] - 0s 1ms/step - loss: 0.0167 - mean_squared_error: 0.0094 - val_loss: 0.0162 - val_mean_squared_error: 0.0091
Epoch 106/150
37/37 [==============================] - 0s 1ms/step - loss: 0.0167 - mean_squared_error: 0.0094 - val_loss: 0.0161 - val_mean_squared_error: 0.0089
Epoch 107/150
37/37 [===============

No mejora

## 2.3. Modelo 3: Red neuronal con 3 capas y regularización Droput 0.5

In [14]:
# Construyo el modelo
modelo3 = models.Sequential()
modelo3.add(layers.Dropout(0.5, input_shape=(X_train.shape[1],)))
modelo3.add(layers.Dense(16, activation='relu'))
modelo3.add(layers.Dropout(0.5))
modelo3.add(layers.Dense(32, activation='relu'))
modelo3.add(layers.Dropout(0.5))
modelo3.add(layers.Dense(64, activation='relu'))
modelo3.add(layers.Dropout(0.5))
modelo3.add(layers.Dense(1, activation='linear'))

# Compilo el modelo
compile(modelo3)

# Model summary
print(modelo3.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout (Dropout)            (None, 10)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 16)                176       
_________________________________________________________________
dropout_1 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_9 (Dense)              (None, 32)                544       
_________________________________________________________________
dropout_2 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 64)                2112      
_________________________________________________________________
dropout_3 (Dropout)          (None, 64)               

In [15]:
#Entreno el modelo con 150 épocas y batch_size de 256
entrenamiento(X_train, y_train, 150, 256, 0.1, modelo3)

#Evaluo el modelo con las muestras de test
evaluacion(X_train, y_train, X_test, y_test, modelo3)

Epoch 1/150
37/37 [==============================] - 1s 4ms/step - loss: 0.6794 - mean_squared_error: 0.6794 - val_loss: 0.1178 - val_mean_squared_error: 0.1178
Epoch 2/150
37/37 [==============================] - 0s 2ms/step - loss: 0.3378 - mean_squared_error: 0.3378 - val_loss: 0.0771 - val_mean_squared_error: 0.0771
Epoch 3/150
37/37 [==============================] - 0s 2ms/step - loss: 0.1994 - mean_squared_error: 0.1994 - val_loss: 0.0492 - val_mean_squared_error: 0.0492
Epoch 4/150
37/37 [==============================] - 0s 2ms/step - loss: 0.1652 - mean_squared_error: 0.1652 - val_loss: 0.0402 - val_mean_squared_error: 0.0402
Epoch 5/150
37/37 [==============================] - 0s 2ms/step - loss: 0.1068 - mean_squared_error: 0.1068 - val_loss: 0.0302 - val_mean_squared_error: 0.0302
Epoch 6/150
37/37 [==============================] - 0s 2ms/step - loss: 0.0846 - mean_squared_error: 0.0846 - val_loss: 0.0292 - val_mean_squared_error: 0.0292
Epoch 7/150
37/37 [===============

37/37 [==============================] - 0s 1ms/step - loss: 0.0224 - mean_squared_error: 0.0224 - val_loss: 0.0156 - val_mean_squared_error: 0.0156
Epoch 52/150
37/37 [==============================] - 0s 1ms/step - loss: 0.0219 - mean_squared_error: 0.0219 - val_loss: 0.0153 - val_mean_squared_error: 0.0153
Epoch 53/150
37/37 [==============================] - 0s 2ms/step - loss: 0.0220 - mean_squared_error: 0.0220 - val_loss: 0.0155 - val_mean_squared_error: 0.0155
Epoch 54/150
37/37 [==============================] - 0s 2ms/step - loss: 0.0220 - mean_squared_error: 0.0220 - val_loss: 0.0153 - val_mean_squared_error: 0.0153
Epoch 55/150
37/37 [==============================] - 0s 2ms/step - loss: 0.0220 - mean_squared_error: 0.0220 - val_loss: 0.0157 - val_mean_squared_error: 0.0157
Epoch 56/150
37/37 [==============================] - 0s 2ms/step - loss: 0.0221 - mean_squared_error: 0.0221 - val_loss: 0.0156 - val_mean_squared_error: 0.0156
Epoch 57/150
37/37 [=====================

37/37 [==============================] - 0s 2ms/step - loss: 0.0178 - mean_squared_error: 0.0178 - val_loss: 0.0135 - val_mean_squared_error: 0.0135
Epoch 102/150
37/37 [==============================] - 0s 2ms/step - loss: 0.0177 - mean_squared_error: 0.0177 - val_loss: 0.0139 - val_mean_squared_error: 0.0139
Epoch 103/150
37/37 [==============================] - 0s 2ms/step - loss: 0.0178 - mean_squared_error: 0.0178 - val_loss: 0.0138 - val_mean_squared_error: 0.0138
Epoch 104/150
37/37 [==============================] - 0s 2ms/step - loss: 0.0178 - mean_squared_error: 0.0178 - val_loss: 0.0137 - val_mean_squared_error: 0.0137
Epoch 105/150
37/37 [==============================] - 0s 2ms/step - loss: 0.0175 - mean_squared_error: 0.0175 - val_loss: 0.0137 - val_mean_squared_error: 0.0137
Epoch 106/150
37/37 [==============================] - 0s 2ms/step - loss: 0.0175 - mean_squared_error: 0.0175 - val_loss: 0.0139 - val_mean_squared_error: 0.0139
Epoch 107/150
37/37 [===============

El comportamiento de este modelo es peor que el de la regularización L1

## 2.4. Modelo 4: Red neuronal con 3 capas, regularización Dropout 0.2 y mayor batch size

In [16]:
# Construyo el modelo
modelo4 = models.Sequential()
modelo4.add(layers.Dropout(0.2, input_shape=(X_train.shape[1],)))
modelo4.add(layers.Dense(16, activation='relu'))
modelo4.add(layers.Dropout(0.2))
modelo4.add(layers.Dense(32, activation='relu'))
modelo4.add(layers.Dropout(0.2))
modelo4.add(layers.Dense(64, activation='relu'))
modelo4.add(layers.Dropout(0.2))
modelo4.add(layers.Dense(1, activation='linear'))

# Compiling the model
modelo4.compile(loss='mean_squared_error',
            optimizer='adam',
            metrics=['mean_squared_error'])

# Model summary
print(modelo4.summary())

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dropout_4 (Dropout)          (None, 10)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 16)                176       
_________________________________________________________________
dropout_5 (Dropout)          (None, 16)                0         
_________________________________________________________________
dense_13 (Dense)             (None, 32)                544       
_________________________________________________________________
dropout_6 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_14 (Dense)             (None, 64)                2112      
_________________________________________________________________
dropout_7 (Dropout)          (None, 64)               

In [17]:
#Entreno el modelo con 150 épocas y batch_size de 512
entrenamiento(X_train, y_train, 150, 512, 0.1, modelo4)

#Evaluo el modelo con las muestras de test
evaluacion(X_train, y_train, X_test, y_test, modelo4)

Epoch 1/150
19/19 [==============================] - 1s 9ms/step - loss: 0.3721 - mean_squared_error: 0.3721 - val_loss: 0.1092 - val_mean_squared_error: 0.1092
Epoch 2/150
19/19 [==============================] - 0s 3ms/step - loss: 0.1279 - mean_squared_error: 0.1279 - val_loss: 0.0538 - val_mean_squared_error: 0.0538
Epoch 3/150
19/19 [==============================] - 0s 3ms/step - loss: 0.0784 - mean_squared_error: 0.0784 - val_loss: 0.0351 - val_mean_squared_error: 0.0351
Epoch 4/150
19/19 [==============================] - 0s 3ms/step - loss: 0.0610 - mean_squared_error: 0.0610 - val_loss: 0.0280 - val_mean_squared_error: 0.0280
Epoch 5/150
19/19 [==============================] - 0s 3ms/step - loss: 0.0519 - mean_squared_error: 0.0519 - val_loss: 0.0253 - val_mean_squared_error: 0.0253
Epoch 6/150
19/19 [==============================] - 0s 3ms/step - loss: 0.0458 - mean_squared_error: 0.0458 - val_loss: 0.0219 - val_mean_squared_error: 0.0219
Epoch 7/150
19/19 [===============

19/19 [==============================] - 0s 2ms/step - loss: 0.0150 - mean_squared_error: 0.0150 - val_loss: 0.0101 - val_mean_squared_error: 0.0101
Epoch 52/150
19/19 [==============================] - 0s 2ms/step - loss: 0.0152 - mean_squared_error: 0.0152 - val_loss: 0.0106 - val_mean_squared_error: 0.0106
Epoch 53/150
19/19 [==============================] - 0s 2ms/step - loss: 0.0148 - mean_squared_error: 0.0148 - val_loss: 0.0104 - val_mean_squared_error: 0.0104
Epoch 54/150
19/19 [==============================] - 0s 2ms/step - loss: 0.0150 - mean_squared_error: 0.0150 - val_loss: 0.0101 - val_mean_squared_error: 0.0101
Epoch 55/150
19/19 [==============================] - 0s 2ms/step - loss: 0.0150 - mean_squared_error: 0.0150 - val_loss: 0.0101 - val_mean_squared_error: 0.0101
Epoch 56/150
19/19 [==============================] - 0s 3ms/step - loss: 0.0146 - mean_squared_error: 0.0146 - val_loss: 0.0100 - val_mean_squared_error: 0.0100
Epoch 57/150
19/19 [=====================

19/19 [==============================] - 0s 3ms/step - loss: 0.0128 - mean_squared_error: 0.0128 - val_loss: 0.0105 - val_mean_squared_error: 0.0105
Epoch 102/150
19/19 [==============================] - 0s 2ms/step - loss: 0.0129 - mean_squared_error: 0.0129 - val_loss: 0.0099 - val_mean_squared_error: 0.0099
Epoch 103/150
19/19 [==============================] - 0s 3ms/step - loss: 0.0128 - mean_squared_error: 0.0128 - val_loss: 0.0102 - val_mean_squared_error: 0.0102
Epoch 104/150
19/19 [==============================] - 0s 2ms/step - loss: 0.0128 - mean_squared_error: 0.0128 - val_loss: 0.0101 - val_mean_squared_error: 0.0101
Epoch 105/150
19/19 [==============================] - 0s 2ms/step - loss: 0.0127 - mean_squared_error: 0.0127 - val_loss: 0.0101 - val_mean_squared_error: 0.0101
Epoch 106/150
19/19 [==============================] - 0s 2ms/step - loss: 0.0127 - mean_squared_error: 0.0127 - val_loss: 0.0099 - val_mean_squared_error: 0.0099
Epoch 107/150
19/19 [===============

Mejora el anterior pero no el primero

## 2.5. Modelo 5: Red neuronal con 4 capas, regularización L1 

In [18]:
# Construyo el modelo igual que el modelo2
modelo5 = models.Sequential()
modelo5.add(layers.Dense(16, input_shape=(X_train.shape[1],), kernel_regularizer=regularizers.l1(0.005), activation='relu'))
modelo5.add(layers.Dense(32, kernel_regularizer=regularizers.l1(0.002), activation='relu'))
modelo5.add(layers.Dense(32, kernel_regularizer=regularizers.l1(0.002), activation='relu'))
modelo5.add(layers.Dense(32, kernel_regularizer=regularizers.l1(0.002), activation='relu'))
modelo5.add(layers.Dense(1, activation='linear'))

# Compilo el modelo
compile(modelo5)

# Model summary
print(modelo5.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_16 (Dense)             (None, 16)                176       
_________________________________________________________________
dense_17 (Dense)             (None, 32)                544       
_________________________________________________________________
dense_18 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_19 (Dense)             (None, 32)                1056      
_________________________________________________________________
dense_20 (Dense)             (None, 1)                 33        
Total params: 2,865
Trainable params: 2,865
Non-trainable params: 0
_________________________________________________________________
None


In [19]:
#Entreno el modelo con 150 épocas y batch_size de 256
entrenamiento(X_train, y_train, 150, 256, 0.1, modelo5)

#Evaluo el modelo con las muestras de test
evaluacion(X_train, y_train, X_test, y_test, modelo5)

Epoch 1/150
37/37 [==============================] - 1s 5ms/step - loss: 1.0247 - mean_squared_error: 0.0901 - val_loss: 0.9016 - val_mean_squared_error: 0.0361
Epoch 2/150
37/37 [==============================] - 0s 1ms/step - loss: 0.8218 - mean_squared_error: 0.0235 - val_loss: 0.7423 - val_mean_squared_error: 0.0149
Epoch 3/150
37/37 [==============================] - 0s 2ms/step - loss: 0.6771 - mean_squared_error: 0.0134 - val_loss: 0.6083 - val_mean_squared_error: 0.0107
Epoch 4/150
37/37 [==============================] - 0s 1ms/step - loss: 0.5504 - mean_squared_error: 0.0110 - val_loss: 0.4897 - val_mean_squared_error: 0.0097
Epoch 5/150
37/37 [==============================] - 0s 1ms/step - loss: 0.4396 - mean_squared_error: 0.0102 - val_loss: 0.3876 - val_mean_squared_error: 0.0095
Epoch 6/150
37/37 [==============================] - 0s 2ms/step - loss: 0.3451 - mean_squared_error: 0.0101 - val_loss: 0.3014 - val_mean_squared_error: 0.0094
Epoch 7/150
37/37 [===============

37/37 [==============================] - 0s 1ms/step - loss: 0.0164 - mean_squared_error: 0.0094 - val_loss: 0.0159 - val_mean_squared_error: 0.0090
Epoch 52/150
37/37 [==============================] - 0s 1ms/step - loss: 0.0164 - mean_squared_error: 0.0094 - val_loss: 0.0159 - val_mean_squared_error: 0.0089
Epoch 53/150
37/37 [==============================] - 0s 1ms/step - loss: 0.0164 - mean_squared_error: 0.0094 - val_loss: 0.0158 - val_mean_squared_error: 0.0088
Epoch 54/150
37/37 [==============================] - 0s 1ms/step - loss: 0.0164 - mean_squared_error: 0.0094 - val_loss: 0.0162 - val_mean_squared_error: 0.0091
Epoch 55/150
37/37 [==============================] - 0s 1ms/step - loss: 0.0164 - mean_squared_error: 0.0094 - val_loss: 0.0158 - val_mean_squared_error: 0.0089
Epoch 56/150
37/37 [==============================] - 0s 2ms/step - loss: 0.0163 - mean_squared_error: 0.0094 - val_loss: 0.0158 - val_mean_squared_error: 0.0089
Epoch 57/150
37/37 [=====================

37/37 [==============================] - 0s 1ms/step - loss: 0.0157 - mean_squared_error: 0.0094 - val_loss: 0.0152 - val_mean_squared_error: 0.0088
Epoch 102/150
37/37 [==============================] - 0s 1ms/step - loss: 0.0158 - mean_squared_error: 0.0094 - val_loss: 0.0152 - val_mean_squared_error: 0.0089
Epoch 103/150
37/37 [==============================] - 0s 1ms/step - loss: 0.0157 - mean_squared_error: 0.0094 - val_loss: 0.0152 - val_mean_squared_error: 0.0088
Epoch 104/150
37/37 [==============================] - 0s 1ms/step - loss: 0.0157 - mean_squared_error: 0.0094 - val_loss: 0.0152 - val_mean_squared_error: 0.0089
Epoch 105/150
37/37 [==============================] - 0s 1ms/step - loss: 0.0157 - mean_squared_error: 0.0094 - val_loss: 0.0153 - val_mean_squared_error: 0.0089
Epoch 106/150
37/37 [==============================] - 0s 1ms/step - loss: 0.0157 - mean_squared_error: 0.0094 - val_loss: 0.0152 - val_mean_squared_error: 0.0089
Epoch 107/150
37/37 [===============

Conseguimos resultados similares al del modelo 2 

## 7.6. Conclusión. Modelo Red Neuronal con datos solo numéricos

Tras las diversas pruebas se selecciona el primer modelo 